In [17]:
import pandas as pd

#read data
train= pd.read_csv('/media/sf_Bioinfo_General/train.csv')
test= pd.read_csv('/media/sf_Bioinfo_General/test.csv')

print(train.shape, test.shape)
train.head()

(7973, 7) (3, 2)


id                                             SMILES  Tg       FFV  \
0   87817                         *CC(*)c1ccccc1C(=O)OCCCCCC NaN  0.374645   
1  106919  *Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5... NaN  0.370410   
2  388772  *Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(... NaN  0.378860   
3  519416  *Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)... NaN  0.387324   
4  539187  *Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N... NaN  0.355470   

         Tc  Density  Rg  
0  0.205667      NaN NaN  
1       NaN      NaN NaN  
2       NaN      NaN NaN  
3       NaN      NaN NaN  
4       NaN      NaN NaN

In [24]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

def smiles_to_fp(smi, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return np.zero((nBits,), dtype=np.uint8)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol,radius,nBits)
    arr = np.zeros((nBits,),dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp,arr)
    return arr

X = np.vstack(train["SMILES"].apply(smiles_to_fp))
X_test = np.vstack(test["SMILES"].apply(smiles_to_fp))

[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerator
[17:07:28] DEPRECATION WARNING: please use MorganGenerat

In [31]:
import xgboost as xgb

mask = train["FFV"].notna()
X_train = X[mask]
y_train = train.loc[mask, "FFV"]

model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=-1, num_parallel_tree=None, ...)

In [30]:
preds = model.predict(X_test)
submission = pd.DataFrame({
    "id": test["id"],
    "FFV": preds
})
submission.to_csv("submission_ffv.csv", index=False)